In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))

1115394


In [3]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
# create mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode('hi there'))
print(decode(encode('hello')))

[46, 47, 1, 58, 46, 43, 56, 43]
hello


In [5]:
# encode entire dataset

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [7]:
# split train and test data
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [17]:
# block size
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(t, context, target)

0 tensor([18]) tensor(47)
1 tensor([18, 47]) tensor(56)
2 tensor([18, 47, 56]) tensor(57)
3 tensor([18, 47, 56, 57]) tensor(58)
4 tensor([18, 47, 56, 57, 58]) tensor(1)
5 tensor([18, 47, 56, 57, 58,  1]) tensor(15)
6 tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
7 tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [18]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"Input: {context.tolist()} Target: {target}")

Input: [57] Target: 1
Input: [57, 1] Target: 46
Input: [57, 1, 46] Target: 47
Input: [57, 1, 46, 47] Target: 57
Input: [57, 1, 46, 47, 57] Target: 1
Input: [57, 1, 46, 47, 57, 1] Target: 50
Input: [57, 1, 46, 47, 57, 1, 50] Target: 53
Input: [57, 1, 46, 47, 57, 1, 50, 53] Target: 60
Input: [1] Target: 58
Input: [1, 58] Target: 46
Input: [1, 58, 46] Target: 43
Input: [1, 58, 46, 43] Target: 56
Input: [1, 58, 46, 43, 56] Target: 43
Input: [1, 58, 46, 43, 56, 43] Target: 1
Input: [1, 58, 46, 43, 56, 43, 1] Target: 41
Input: [1, 58, 46, 43, 56, 43, 1, 41] Target: 39
Input: [17] Target: 26
Input: [17, 26] Target: 15
Input: [17, 26, 15] Target: 17
Input: [17, 26, 15, 17] Target: 10
Input: [17, 26, 15, 17, 10] Target: 0
Input: [17, 26, 15, 17, 10, 0] Target: 32
Input: [17, 26, 15, 17, 10, 0, 32] Target: 53
Input: [17, 26, 15, 17, 10, 0, 32, 53] Target: 1
Input: [57] Target: 58
Input: [57, 58] Target: 6
Input: [57, 58, 6] Target: 1
Input: [57, 58, 6, 1] Target: 61
Input: [57, 58, 6, 1, 61] Tar

In [32]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)    # B,T,C
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss


    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape, loss)

idx = torch.zeros((1, 1), dtype=torch.long)
generated = model.generate(idx, max_new_tokens=100)
decoded = decode(generated[0].tolist())
print(decoded)

torch.Size([32, 65]) tensor(4.8865, grad_fn=<NllLossBackward0>)

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


In [34]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [44]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.4933512210845947


In [47]:
idx = torch.zeros((1, 1), dtype=torch.long)
generated = model.generate(idx, max_new_tokens=400)
decoded = decode(generated[0].tolist())
print(decoded)


Tonothivind d y thon t hyounepssd!
Fourepl bet hee wit asblcak ut handathit


BE ay l. ll tha hacher l, y
And.

O:
Fosth ff s ss and't tor sonin wnn t h plercend the, headisony wstisaur tene'le pay tucuio y aveencer
O:
GLowipase nisthiry II ENo sose.
H:
3sseth ws mequr nd thauk h thre a g tode IV:
Macalkilvorith onows toum athessth INumy ak mellope!

JUSARIth;
GENTHAbrt yowd twos, t con. n,-Maveru


### Self-attention

In [48]:
torch.manual_seed(42)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [49]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, dim=0)